In [1]:
#Make imports
import numpy as np
import re
import string
import pandas as pd
import tensorflow as tf

In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [3]:
def preprocess(text):
  text = ''.join(ch for ch in text if ch not in string.punctuation)
  text = text.lower()
  text = re.sub(r'\d','',text)
  text = re.sub(r'\s+',' ',text)
  text = text.strip()
  return text

In [4]:
from datasets import load_dataset
corpus = load_dataset("cfilt/iitb-english-hindi")
data=corpus["validation"]["translation"]+corpus["train"]["translation"]+corpus["test"]["translation"]
Data=pd.DataFrame(data)
hindi_sentences=Data["hi"]
english_sentences=Data["en"]
hindi_sentences = ['<START> ' + re.sub('[a-zA-Z]','',preprocess(hi)) + ' <END>' for hi in Data["hi"]]

Using custom data configuration cfilt--iitb-english-hindi-911387c6837f8b91
Found cached dataset parquet (C:/Users/joshi/.cache/huggingface/datasets/cfilt___parquet/cfilt--iitb-english-hindi-911387c6837f8b91/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
print(len(english_sentences), len(hindi_sentences))
print()
english_sentences[:3], hindi_sentences[:3]

1662110 1662110



(0    Students of the Dattatreya city Municipal corp...
 1    With encouragement from Principal Sandhya Medp...
 2    Rajesh Gavre, the President of the MNPA teache...
 Name: en, dtype: object,
 ['<START> महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला दत्तगढ़ बनाकर अपनी कल्पनाशक्ति का परिचय दिया। <END>',
  '<START> प्रधानाध्यापक संध्या मेडपल्लीवार के प्रोत्साहित करने पर शिक्षकों व विद्यार्थियों ने मिट्टïी से किले का निर्माण किया। <END>',
  '<START> मनपा शिक्षक संघ के अध्यक्ष राजेश गवरे ने स्कूल को भेंट देकर सराहना की। <END>'])

In [6]:
#Some parameters
vocab_size = 10000
total_sentences = 25000
maxlen = 10
epochs = 15
validation_split = 0.05

In [7]:
en_data = []
hi_data = []

cnt = 0

for (en,hi) in zip(english_sentences, hindi_sentences):
  l = min(len(en.split()), len(hi.split()))
  if l <= maxlen:
    en_data.append(en)
    hi_data.append(hi)
    cnt += 1
  if cnt == total_sentences:
    break

In [8]:
#Tokenize the texts and convert to sequences
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
en_tokenizer.fit_on_texts(en_data)
en_sequences = en_tokenizer.texts_to_sequences(en_data)

hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
hi_tokenizer.fit_on_texts(hi_data)
hi_sequences = hi_tokenizer.texts_to_sequences(hi_data)

english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(hi_tokenizer.word_index) + 1
print("English Vocab Size: ", english_vocab_size)
print("Hindi Vocab Size: ", hindi_vocab_size)

English Vocab Size:  3641
Hindi Vocab Size:  2382


In [9]:
#Prepare encoder data
encoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(en_sequences, maxlen=maxlen, padding='post')

In [10]:
#Prepare decoder data
decoder_inputs = []
decoder_outputs = []

for hi in hi_sequences:
  decoder_inputs.append(hi[:-1])
  decoder_outputs.append(hi[1:])

decoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs, maxlen=maxlen, padding='post')
decoder_outputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs, maxlen=maxlen, padding='post')

In [11]:
# Training and Testing split

split = int(0.95 * total_sentences)

X_train = [encoder_inputs[:split], decoder_inputs[:split]]
y_train = decoder_outputs[:split]

X_test = en_data[:split]
y_test = hi_data[:split]

print(X_train[0].shape, X_train[1].shape, y_train.shape)

(23750, 10) (23750, 10) (23750, 10)


In [51]:
#Define LSTM model
d_model = 256

#Encoder
inputs = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(english_vocab_size, d_model, mask_zero=True)(inputs)
_,state_h,state_c = tf.keras.layers.LSTM(d_model,activation='relu',return_state=True)(x)

#Decoder
targets = tf.keras.layers.Input(shape=(None,))
embedding_layer = tf.keras.layers.Embedding(hindi_vocab_size, d_model, mask_zero=True)
x = embedding_layer(targets)
decoder_lstm = tf.keras.layers.LSTM(d_model,activation='relu',return_sequences=True, return_state=True)
x,_,_ = decoder_lstm(x, initial_state=[state_h, state_c])
dense1 = tf.keras.layers.Dense(hindi_vocab_size, activation='softmax')
x = dense1(x)

model = tf.keras.models.Model(inputs=[inputs, targets],outputs=x)
model.summary()

loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, None, 256)    932096      ['input_15[0][0]']               
                                                                                                  
 embedding_7 (Embedding)        (None, None, 256)    609792      ['input_16[0][0]']               
                                                                                           

In [48]:
#Save model after each epoch
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='en-hi.h5',
    monitor='val_accuracy',
    mode='max'
)

In [49]:
model.fit(X_train, y_train, epochs=10, validation_split=validation_split, callbacks=[save_model_callback, tf.keras.callbacks.TerminateOnNaN()])

Epoch 1/10
706/706 [==============================] - 128s 182ms/step - loss: 0.6562 - accuracy: 0.6925 - val_loss: 0.3396 - val_accuracy: 0.8053
Epoch 2/10
706/706 [==============================] - 122s 173ms/step - loss: 0.4702 - accuracy: 0.7837 - val_loss: 0.1894 - val_accuracy: 0.8924
Epoch 3/10
706/706 [==============================] - 139s 197ms/step - loss: 0.3362 - accuracy: 0.8456 - val_loss: 0.1043 - val_accuracy: 0.9356
Epoch 4/10
706/706 [==============================] - 136s 192ms/step - loss: 0.2424 - accuracy: 0.8897 - val_loss: 0.0514 - val_accuracy: 0.9715
Epoch 5/10
706/706 [==============================] - 123s 174ms/step - loss: 0.1855 - accuracy: 0.9190 - val_loss: 0.0319 - val_accuracy: 0.9853
Epoch 6/10
706/706 [==============================] - 125s 178ms/step - loss: 0.1575 - accuracy: 0.9325 - val_loss: 0.0185 - val_accuracy: 0.9911
Epoch 7/10
706/706 [==============================] - 130s 184ms/step - loss: 0.1402 - accuracy: 0.9389 - val_loss: 0.0131 -

In [54]:
#Retrieve previously saved stuff
saved_model = tf.keras.models.load_model('en-hi.h5')
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, None, 256)    932096      ['input_15[0][0]']               
                                                                                                  
 embedding_7 (Embedding)        (None, None, 256)    609792      ['input_16[0][0]']               
                                                                                           

In [55]:
inputs = saved_model.get_layer('input_11').output
_,state_h,state_c = saved_model.get_layer('lstm_4').output
targets = saved_model.get_layer('input_12').output
embedding_layer = saved_model.get_layer('embedding_5')
decoder_lstm = saved_model.get_layer('lstm_5')
dense1 = saved_model.get_layer('dense_2')

In [56]:
#Inference Model

#Encoder
encoder = tf.keras.models.Model(inputs, [state_h, state_c])

#Decoder
decoder_input_h = tf.keras.layers.Input(shape=(d_model,))
decoder_input_c = tf.keras.layers.Input(shape=(d_model,))
x = embedding_layer(targets)
x, decoder_output_h, decoder_output_c = decoder_lstm(x, initial_state=[decoder_input_h, decoder_input_c])
x = dense1(x)
decoder = tf.keras.models.Model([targets] + [decoder_input_h, decoder_input_c], 
                                [x] + [decoder_output_h, decoder_output_c])

In [57]:
def predict_sentence(en_input):
  input_seq = en_tokenizer.texts_to_sequences([en_input])

  next_h, next_c = encoder.predict(input_seq)

  curr_token = np.zeros(1)
  curr_token[0] = hi_tokenizer.word_index['<START>']

  pred_sentence = ''

  for i in range(maxlen):
    output, next_h, next_c = decoder.predict([curr_token] + [next_h , next_c])
    next_token = np.argmax(output[0, 0, :])
    next_word = hi_tokenizer.index_word[next_token]
    if next_word == '<END>':
      break
    else:
      pred_sentence += ' ' + next_word
      curr_token[0] = next_token

  return pred_sentence

In [61]:
l = len(X_test[0].split())
print(X_test[0])
pred_sentence = predict_sentence(X_test[0])
print(pred_sentence)

Ramesh Saatpute examined the fort.
रमेश सातपुते ने किले का निरीक्षण किया


In [62]:
#Testing and Analysis
import nltk

candidates = []
references = []

import googletrans 
translator = googletrans.Translator()

ctr = 20 
i = 0

while ctr>0:
  l = len(X_test[i].split())
  if l<=maxlen:
    pred_sentence = predict_sentence(X_test[i])
    candidates.append(pred_sentence.split())
    google_translated_sentence = translator.translate(X_test[i], dest='hi').text
    print()
    references.append([y_test[i].split()[1:-1], google_translated_sentence.split()])

    ctr -= 1
  i += 1

print("BLEU Score is : ",nltk.translate.bleu_score.corpus_bleu(references, candidates))

BLEU Score is : 40.72
